In [4]:
from utils import *

check_tensorflow_gpu()
K.clear_session()

proxy = SpatiotemporalCO2()

proxy.num_epochs = 200
proxy.batch_size = 16
proxy.verbose = 1

Checking Tensorflow Version:
Tensorflow built with CUDA? True
Tensorflow version: 2.10.0
# GPU available: 1
CUDA: 64_112 | cuDNN: 64_8
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
proxy.load_data()
proxy.process_data(subsample=600)

X: (1000, 64, 64, 4) | y: (1000, 60, 64, 64, 2)
normalized - X: (1000, 64, 64, 4) | y: (1000, 60, 64, 64, 2)
Subsampling data for 600 samples ...
Train - X: (450, 64, 64, 4) | y: (450, 60, 64, 64, 2)
Test  - X: (150, 64, 64, 4) | y: (150, 60, 64, 64, 2)


In [6]:
model0 = proxy.make_model()
model, fit = proxy.train(model0)

# Parameters: 458,514
Epoch 1/200


ResourceExhaustedError: Graph execution error:

Detected at node 'model_1/conv_lstm2d_1/transpose_1' defined at (most recent call last):
    File "c:\Users\Misael Morales\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\Misael Morales\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\Misael Morales\AppData\Roaming\Python\Python39\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\Misael Morales\AppData\Local\Programs\Python\Python39\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "C:\Users\Misael Morales\AppData\Roaming\Python\Python39\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "C:\Users\Misael Morales\AppData\Roaming\Python\Python39\site-p

In [ ]:
y_train_pred = model.predict(proxy.X_train)
y_test_pred  = model.predict(proxy.X_test)
print('Train pred: {} | Test pred: {}'.format(y_train_pred.shape, y_test_pred.shape))

In [ ]:
k = 78
c = 1

fig, axs = plt.subplots(2, 12, figsize=(20,4))
for j in range(12):
    axs[0,j].imshow(proxy.y_train[k,j*5,:,:,c], 'jet')
    axs[1,j].imshow(y_train_pred[k,j*5,:,:,c], 'jet')
    axs[0,j].set(title='t={}'.format(j*5))
    for i in range(2):
        axs[i,j].set(xticks=[], yticks=[])
axs[0,0].set(ylabel='True'); axs[1,0].set(ylabel='Pred')
plt.show()

***
# END

In [21]:
# Define proxy model by blocks
global_reg = 1e-4

# Convolutional block (Encoder)
def conv_block(filt, inp, kern=(3,3), reg=global_reg):
    x = Conv2D(filters=filt, kernel_size=kern, padding='same', activation=LeakyReLU(alpha=0.3))(inp)
    x = Conv2D(filters=filt, kernel_size=kern, padding='same', kernel_regularizer=regularizers.l2(reg))(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.3)(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    return x

# Recurrent block
def rnn_block(units, inp, drop=0.2):
    x = Flatten()(inp)
    x = RepeatVector(n=5*4*4)(x)
    x = GRU(units=units, return_sequences=True, dropout=drop)(x)
    x = Reshape((5,4,4, x.shape[-1]))(x)
    return x

# Transpose Convolutional block (Decoder)
def convT_block(filt, stride, inp, kern=(3,3,3), reg=global_reg):
    x = Conv3DTranspose(filters=filt, kernel_size=kern, padding='same', strides=1,      activation=LeakyReLU(alpha=0.3))(inp)
    x = Conv3DTranspose(filters=filt, kernel_size=kern, padding='same', strides=stride, activation=LeakyReLU(alpha=0.3))(x)
    x = Conv3DTranspose(filters=filt, kernel_size=kern, padding='same', kernel_regularizer=regularizers.l2(reg))(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.3)(x)
    return x

# Output block
def output_block(filt, inp, kern=(3,3,3)):
    x = Conv3D(filters=filt, kernel_size=kern, padding='same', activation='sigmoid')(inp)
    return x

# Define CNN-RNN forward proxy model
def make_proxy():
    K.clear_session()

    # Input layer
    inp = Input(shape=(64,64,4))

    # Encoder block
    x = conv_block(filt=8,  inp=inp)
    x = conv_block(filt=16, inp=x)
    x = conv_block(filt=32, inp=x)
    x = conv_block(filt=64, inp=x)

    # Recurrent block
    x = rnn_block(units=128, inp=x)

    # Decoder block
    x = convT_block(filt=64, stride=(2,2,2), inp=x)
    x = convT_block(filt=32, stride=(2,2,2), inp=x)
    x = convT_block(filt=16, stride=(3,2,2), inp=x)
    x = convT_block(filt=8,  stride=(1,2,2), inp=x)

    # Output block
    out = output_block(filt=2, inp=x)

    proxy_model = Model(inp, out)
    return proxy_model

model = make_proxy()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 64, 4)]       0         
                                                                 
 conv2d (Conv2D)             (None, 64, 64, 8)         296       
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 8)         584       
                                                                 
 batch_normalization (BatchN  (None, 64, 64, 8)        32        
 ormalization)                                                   
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 64, 64, 8)         0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 8)        0         
 )                                                           

In [23]:
model.compile(optimizer='adam', loss=proxy.criterion, metrics=['mse'])
fit = model.fit(proxy.X_train, proxy.y_train,
                epochs = 100,
                batch_size = 30,
                validation_split=0.2,
                shuffle = True,
                verbose = 1)

Epoch 1/100
12/12 [==============================] - 12s 694ms/step - loss: 0.1010 - mse: 0.0195 - val_loss: 0.1584 - val_mse: 0.0689
Epoch 2/100
12/12 [==============================] - 7s 551ms/step - loss: 0.0887 - mse: 0.0184 - val_loss: 0.1270 - val_mse: 0.0403
Epoch 3/100
12/12 [==============================] - 7s 591ms/step - loss: 0.0726 - mse: 0.0188 - val_loss: 0.0953 - val_mse: 0.0195
Epoch 4/100
12/12 [==============================] - 8s 680ms/step - loss: 0.0565 - mse: 0.0191 - val_loss: 0.0896 - val_mse: 0.0204
Epoch 5/100
12/12 [==============================] - 8s 634ms/step - loss: 0.0524 - mse: 0.0188 - val_loss: 0.0862 - val_mse: 0.0213
Epoch 6/100
12/12 [==============================] - 7s 569ms/step - loss: 0.0506 - mse: 0.0185 - val_loss: 0.0910 - val_mse: 0.0241
Epoch 7/100
12/12 [==============================] - 6s 545ms/step - loss: 0.0498 - mse: 0.0184 - val_loss: 0.0617 - val_mse: 0.0194
Epoch 8/100
12/12 [==============================] - 7s 605ms/step -